In [8]:
import os
import selenium.webdriver
import csv
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
from concurrent.futures import ThreadPoolExecutor, wait

In [9]:
url1= "https://walmart.com/ip"

url_sets=["https://www.walmart.com/browse/personal-care/electric-toothbrushes/1005862_1007221_2709578_4710",
         "https://www.walmart.com/browse/personal-care/razors/1005862_1007220_1007399",
         "https://www.walmart.com/browse/personal-care/deodorants-antiperspirants/1005862_1001487",
         "https://www.walmart.com/browse/personal-care/body-wash-cleansers/1005862_1071969_1074364",
         "https://www.walmart.com/browse/health/multivitamins/976760_1005863_1699667",
         "https://www.walmart.com/browse/health/mood-stress-support-supplements/976760_1005863_9579127",
         "https://www.walmart.com/browse/health/health-monitors/976760_1005860_1161882",
         "https://www.walmart.com/browse/protein-fitness/976760_1166769"]

categories=["Oral","Razors","Deos","Bath","Vitamins",
           "supplements","Health_monitors","Proteins"]

In [12]:
def run_process(page_number,url_sets):
    for i in range(len(url_sets)):
        top_n= ["1","2","3","4","5","6","7","8","9","10"]
        url_cat=url_sets[i]+"?page="+top_n[page_number]
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        driver= webdriver.Chrome(executable_path='/Users/bhaskar/Documents/chromedriver',options=option)
        driver.get(url_cat)
        body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        driver.quit()
        soupBody_cat = BeautifulSoup(body_cat)
        for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

            final_results.append(tmp['data-id'])

In [13]:

futures = []
final_results = []
with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process,number,url_sets)
        )
wait(futures)

DoneAndNotDoneFutures(done={<Future at 0x7f8d62a0d250 state=finished returned NoneType>, <Future at 0x7f8d62a06e50 state=finished returned NoneType>, <Future at 0x7f8d62a0da90 state=finished returned NoneType>, <Future at 0x7f8d62a3b090 state=finished returned NoneType>, <Future at 0x7f8d62a3b0d0 state=finished returned NoneType>, <Future at 0x7f8d629afd10 state=finished returned NoneType>, <Future at 0x7f8d62a43510 state=finished returned NoneType>, <Future at 0x7f8d62a0dd50 state=finished returned NoneType>, <Future at 0x7f8d62954b50 state=finished returned NoneType>, <Future at 0x7f8d62a3bfd0 state=finished returned NoneType>}, not_done=set())

In [14]:
len(final_results)

2472

top_n= ["1","2","3","4","5","6","7","8","9","10"]
final_results = []
for i in range(len(top_n)):
    url_cat="https://www.walmart.com/browse/electronics/all-laptop-computers/3944_3951_1089430_132960"+"?page="+top_n[i]
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver= webdriver.Chrome(executable_path='C:/chromedriver.exe',options=option)
    driver.get(url_cat)
    body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
    driver.quit()
    soupBody_cat = BeautifulSoup(body_cat)
    for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

        final_results.append(tmp['data-id'])

In [15]:
len(final_results)

codelist=final_results

In [16]:
codelist[0]

'2YT8ZV7Y1Z49'

In [17]:
def run_process2(i,codelist):
    #creating a list without the place taken in the first loop
    print(i)
    item_wlmt=codelist[i]
    url2=url1+"/"+item_wlmt
    print(url2)
    option = webdriver.ChromeOptions()
    option.add_argument('headless')

    try:
        driver= webdriver.Chrome(executable_path='/Users/bhaskar/Documents/chromedriver',options=option) # Chrome driver is being used.
        print ("Requesting URL: " + url2)
        driver.get(url2)   # URL requested in browser.
        print ("Webpage found ...")
        time.sleep(5)
        # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        print("Closing Chrome ...") # No more usage needed.
        driver.quit() 				# Browser Closed.

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
        #print(soupBody)

        h1ProductName = soupBody.find("h1", {"class": "prod-ProductTitle prod-productTitle-buyBox font-bold"})
        divProductDesc = soupBody.find("div", {"class": "about-desc about-product-description xs-margin-top"})
        liProductBreadcrumb_parent = soupBody.find("li", {"data-automation-id": "breadcrumb-item-0"})
        liProductBreadcrumb_active = soupBody.find("li", {"class": "breadcrumb active"})
        spanProductPrice = soupBody.find("span", {"class": "price-group"})
        spanProductRating = soupBody.find("span", {"itemprop": "ratingValue"})
        spanProductRating_count = soupBody.find("span", {"class": "stars-reviews-count-node"})
            
        ################# exceptions #########################
        if divProductDesc is None:
            divProductDesc="Not Available"
        else:
            divProductDesc=divProductDesc
                
        if liProductBreadcrumb_parent is None:
            liProductBreadcrumb_parent="Not Available"
        else:
            liProductBreadcrumb_parent=liProductBreadcrumb_parent
                
        if liProductBreadcrumb_active is None:
            liProductBreadcrumb_active="Not Available"
        else:
            liProductBreadcrumb_active=liProductBreadcrumb_active
                
        if spanProductPrice is None:
            spanProductPrice="NA"
        else:
            spanProductPrice=spanProductPrice

        if spanProductRating is None or spanProductRating_count is None:
            spanProductRating=0.0
            spanProductRating_count="0 ratings"
            #print(spanProductBreadcrumb_active.text)
        else:
            spanProductRating=spanProductRating.text
            spanProductRating_count=spanProductRating_count.text


            ### Recommended Products
        reco_prods=[]
        for tmp in soupBody.find_all('a', {'class':'tile-link-overlay u-focusTile'}):
            reco_prods.append(tmp['data-product-id'])

        if len(reco_prods)==0:
            reco_prods=["Not available"]
        else:
            reco_prods=reco_prods
        print(len(reco_prods))
        WLMTData.append([codelist[i],h1ProductName.text, divProductDesc.text,url2,
                         liProductBreadcrumb_parent.text,liProductBreadcrumb_active.text,
                         spanProductPrice.text,spanProductRating,spanProductRating_count,reco_prods])
    except Exception as e:
        print (str(e))

In [ ]:
futures2 = []
WLMTData = [["Product_code","Product_name", "Product_description","Product_URL",
                 "Breadcrumb_parent","Breadcrumb_active",
                "Product_price", "Rating_Value","Rating_Count","Recommended_Prods"]]
with ThreadPoolExecutor() as executor:
    for number in range(len(codelist)):
        futures2.append(
            executor.submit(run_process2,number,codelist)
        )
wait(futures2)

01
https://walmart.com/ip/2YT8ZV7Y1Z49

https://walmart.com/ip/1I8FI39V72PV
2
https://walmart.com/ip/6XG88B1NZMNB
3
https://walmart.com/ip/47XLQASR4C88
45
https://walmart.com/ip/0U3YGN4RJB8T

https://walmart.com/ip/62NAL3OAYMDD
6
https://walmart.com/ip/2TN0IC0PAINM7
8

https://walmart.com/ip/6MXK34G8P79H
https://walmart.com/ip/31SXJANX5JBP
9
https://walmart.com/ip/4E0MA760QN5V
1011

https://walmart.com/ip/1PB0ZJHYIJ40
https://walmart.com/ip/53V4NM0KP2DK
12
https://walmart.com/ip/23QVUVRLJDV1
13
https://walmart.com/ip/12JB22DMYZWG
14
https://walmart.com/ip/6LU4EVHGE95S
1516
https://walmart.com/ip/3DDRU7CJC46T

https://walmart.com/ip/3ZE3CO8UX0CJ
17
https://walmart.com/ip/5TDIJMGBSZUV
18
https://walmart.com/ip/2C28YUPHV3UN
19
https://walmart.com/ip/1SLP0ZRCUWSH
20
https://walmart.com/ip/56UQIZX6X0PS
21
22https://walmart.com/ip/7ERUOBKQF075
23

https://walmart.com/ip/4XPVRY0B04R9
https://walmart.com/ip/7GHFLVWR8INN24

https://walmart.com/ip/3WV6PNTBC4HL
25
https://walmart.com/ip/3M2QJUUJP

https://walmart.com/ip/5FVEVX4FM5B9https://walmart.com/ip/1VSXX7X2TFZS
3025
71
https://walmart.com/ip/4Y06I6J3EYKZ


7230
https://walmart.com/ip/6VTLWECQ5KFC30
73
https://walmart.com/ip/4619IAHLTFP4
10

74
75
20
76
https://walmart.com/ip/1AUYFEBB568O
30
29
77
https://walmart.com/ip/3QMA3TJ87OUR
https://walmart.com/ip/2FOM4Q3P075K
78

https://walmart.com/ip/2Y5HNCSBLB54https://walmart.com/ip/1M8P0MOOE1GB

29
15
79
https://walmart.com/ip/4CGVX8C8R5Q9
80
https://walmart.com/ip/1ZTY3SXRN8IO
30
81
https://walmart.com/ip/5WZOEUCQMNPC
30
82
https://walmart.com/ip/15BXMHDZ4Y8J
Requesting URL: https://walmart.com/ip/2RWTCU9KSNJ9Requesting URL: https://walmart.com/ip/2S5O8OKO03RL

30
83
https://walmart.com/ip/6IE44W82BWSK
25
84
https://walmart.com/ip/0VMIR4IKE5QS
Requesting URL: https://walmart.com/ip/76A87ZBVQYRV
Requesting URL: https://walmart.com/ip/4P8S367KE1JP
Requesting URL: https://walmart.com/ip/5BKCRKW980TE
Requesting URL: https://walmart.com/ip/3UTHU1T7755E
Requesting URL: https://walm

for pg in range(len(url_sets)):
    # number of pages per category
    top_n= ["1","2","3","4","5","6","7","8","9","10"]
    url_category=url_sets[pg]
    print("Category:",categories[pg])
    final_results = []
    for i_1 in range(len(top_n)):
        print("Page number within category:",i_1)
        url_cat=url_category+"?page="+top_n[i_1]
        driver= webdriver.Chrome(executable_path='/Users/bhaskar/Documents/chromedriver.exe')
        driver.get(url_cat)
        body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        driver.quit()
        soupBody_cat = BeautifulSoup(body_cat)
        
        
        for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

            final_results.append(tmp['data-id'])
    codelist=final_results
    print("Total number of prods:",len(codelist))
    # base URL for product page
    url1= "https://walmart.com/ip"

    # Data Headers
    WLMTData = [["Product_code","Product_name", "Product_description","Product_URL",
                 "Breadcrumb_parent","Breadcrumb_active",
                "Product_price", "Rating_Value","Rating_Count","Recommended_Prods"]]
    for i in range(len(codelist)):
        #creating a list without the place taken in the first loop
        print(i)
        item_wlmt=codelist[i]
        url2=url1+"/"+item_wlmt
        #print(url2)
        
        try:
            driver= webdriver.Chrome(executable_path='C:/Drivers/chromedriver.exe') # Chrome driver is being used.
            print ("Requesting URL: " + url2)

            driver.get(url2)   # URL requested in browser.
            print ("Webpage found ...")
            time.sleep(5)
            # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
            body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
            print("Closing Chrome ...") # No more usage needed.
            driver.quit() 				# Browser Closed.

            print("Getting data from DOM ...")
            soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
            #print(soupBody)

            h1ProductName = soupBody.find("h1", {"class": "prod-ProductTitle prod-productTitle-buyBox font-bold"})
            divProductDesc = soupBody.find("div", {"class": "about-desc about-product-description xs-margin-top"})
            liProductBreadcrumb_parent = soupBody.find("li", {"data-automation-id": "breadcrumb-item-0"})
            liProductBreadcrumb_active = soupBody.find("li", {"class": "breadcrumb active"})
            spanProductPrice = soupBody.find("span", {"class": "price-group"})
            spanProductRating = soupBody.find("span", {"itemprop": "ratingValue"})
            spanProductRating_count = soupBody.find("span", {"class": "stars-reviews-count-node"})
            
            ################# exceptions #########################
            if divProductDesc is None:
                divProductDesc="Not Available"
            else:
                divProductDesc=divProductDesc
                
            if liProductBreadcrumb_parent is None:
                liProductBreadcrumb_parent="Not Available"
            else:
                liProductBreadcrumb_parent=liProductBreadcrumb_parent
                
            if liProductBreadcrumb_active is None:
                liProductBreadcrumb_active="Not Available"
            else:
                liProductBreadcrumb_active=liProductBreadcrumb_active
                
            if spanProductPrice is None:
                spanProductPrice="NA"
            else:
                spanProductPrice=spanProductPrice

            if spanProductRating is None or spanProductRating_count is None:
                spanProductRating=0.0
                spanProductRating_count="0 ratings"
            #print(spanProductBreadcrumb_active.text)
            else:
                spanProductRating=spanProductRating.text
                spanProductRating_count=spanProductRating_count.text


            ### Recommended Products
            reco_prods=[]
            for tmp in soupBody.find_all('a', {'class':'tile-link-overlay u-focusTile'}):
                reco_prods.append(tmp['data-product-id'])

            if len(reco_prods)==0:
                reco_prods=["Not available"]
            else:
                reco_prods=reco_prods
            print(len(reco_prods))
            WLMTData.append([codelist[i],h1ProductName.text, divProductDesc.text,url2,
                            liProductBreadcrumb_parent.text,liProductBreadcrumb_active.text,
                            spanProductPrice.text,spanProductRating,spanProductRating_count,reco_prods])




            except Exception as e:
                print (str(e))

#WLMTData        

In [ ]:
df=pd.DataFrame(WLMTData)
df.columns = df.iloc[0]
df=df.drop(df.index[0])
df

In [ ]:
# Output File for FlightsData. This file will have the data in comma separated form.
df.to_csv('personal_care.csv',index=False)